## Домашняя работа №8 Идрисова Руслана

### Задание 1

Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

#### Решение

In [42]:
#Основные импорты
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [43]:
# Загрузим датасет IRIS из sklearn
X, y = load_iris(return_X_y=True)

In [44]:
#разделим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

In [45]:
#функция для подсчета евклидовой метрики
def e_metrics(x1, x2):
    
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])
        
    distance = np.sum(np.square(x1 - x2))

    return np.sqrt(distance)

In [46]:
def knn(x_train, y_train, x_test, k):
    
    answers = [] #подготовим для ответов
    for x in x_test: #проходим по всем тестовым объектам
        test_distances = [] #задаем список их дистанций
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1 #берем 1-й объект из кортежа test_distances (это и будет класс) и добавляем 1 голос за этот класс

        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
        
    return answers

In [47]:
#функция вычисления точности
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [72]:
k = 3

y_pred = knn(X_train, y_train, X_test, k)

print(f'Точность алгоритма при k = {k}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 3: 0.967


Реализуем алгоритм PCA

In [49]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res


X = standard_scale(X)

In [61]:
def covariance(x, y):
    return np.sum(x * y) / (len(x) - 1)

print(f'Ковариация 0 признака (дисперсия 0 признака) {covariance(X[:, 0], X[:, 0])}')
print(f'Ковариация 1 признака (дисперсия 1 признака) {covariance(X[:, 1], X[:, 1])}')
print(f'Ковариация 2 признака (дисперсия 2 признака) {covariance(X[:, 2], X[:, 2])}')
print(f'Ковариация 3 признака (дисперсия 3 признака) {covariance(X[:, 3], X[:, 3])}')

cov = np.cov(X.T)
cov

Ковариация 0 признака (дисперсия 0 признака) 1.0067114093959733
Ковариация 1 признака (дисперсия 1 признака) 1.0067114093959741
Ковариация 2 признака (дисперсия 2 признака) 1.006711409395973
Ковариация 3 признака (дисперсия 3 признака) 1.0067114093959733


array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343066],
       [-0.11835884,  1.00671141, -0.43131554, -0.36858315],
       [ 0.87760447, -0.43131554,  1.00671141,  0.96932762],
       [ 0.82343066, -0.36858315,  0.96932762,  1.00671141]])

In [51]:
X.T @ X / 9

array([[16.66666667, -1.9594964 , 14.5292296 , 13.6323521 ],
       [-1.9594964 , 16.66666667, -7.14066841, -6.10209888],
       [14.5292296 , -7.14066841, 16.66666667, 16.04775719],
       [13.6323521 , -6.10209888, 16.04775719, 16.66666667]])

In [52]:
eig_val, eig_vecs = np.linalg.eig(X.T @ X)
display(eig_val, eig_vecs)

array([437.77467248, 137.10457072,  22.01353134,   3.10722546])

array([[ 0.52106591, -0.37741762, -0.71956635,  0.26128628],
       [-0.26934744, -0.92329566,  0.24438178, -0.12350962],
       [ 0.5804131 , -0.02449161,  0.14212637, -0.80144925],
       [ 0.56485654, -0.06694199,  0.63427274,  0.52359713]])

In [53]:
vec0 = eig_vecs[:, 0]
vec1 = eig_vecs[:, 1]
vec2 = eig_vecs[:, 2]
vec3 = eig_vecs[:, 3]
vec0, vec1, vec2, vec3

(array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]),
 array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]),
 array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]),
 array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))

In [54]:
#оценим объем потерянной (и сохраненной) информации
eig_sum = sum(eig_val)
[(i / eig_sum) * 100 for i in sorted(eig_val, reverse=True)]

[72.96244541329986, 22.85076178670178, 3.668921889282875, 0.5178709107154793]

Таким образом, наш больший вектор описывает 72.96%, второй по объему - 22.85%, а оставшиеся два в сумме, соответственно, 4.19%. Отбросив последние два меньших вектора и спроецировав данные на первые два больших, мы потеряем меньше 5% информации.

Теперь нужно спроецировать данные на векторы.

In [57]:
v0 = vec0
Z1 = np.dot(X, v0)
v1 = vec1
Z2 = np.dot(X, v1)
Z1, Z2

(array([-2.26470281, -2.08096115, -2.36422905, -2.29938422, -2.38984217,
        -2.07563095, -2.44402884, -2.23284716, -2.33464048, -2.18432817,
        -2.1663101 , -2.32613087, -2.2184509 , -2.6331007 , -2.1987406 ,
        -2.26221453, -2.2075877 , -2.19034951, -1.898572  , -2.34336905,
        -1.914323  , -2.20701284, -2.7743447 , -1.81866953, -2.22716331,
        -1.95184633, -2.05115137, -2.16857717, -2.13956345, -2.26526149,
        -2.14012214, -1.83159477, -2.61494794, -2.44617739, -2.10997488,
        -2.2078089 , -2.04514621, -2.52733191, -2.42963258, -2.16971071,
        -2.28647514, -1.85812246, -2.5536384 , -1.96444768, -2.13705901,
        -2.0697443 , -2.38473317, -2.39437631, -2.22944655, -2.20383344,
         1.10178118,  0.73133743,  1.24097932,  0.40748306,  1.0754747 ,
         0.38868734,  0.74652974, -0.48732274,  0.92790164,  0.01142619,
        -0.11019628,  0.44069345,  0.56210831,  0.71956189, -0.0333547 ,
         0.87540719,  0.35025167,  0.15881005,  1.2

In [60]:
#сверим с библиотечным PCA - всё совпало
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)
X_reduced

array([[-2.26470281,  0.4800266 ],
       [-2.08096115, -0.67413356],
       [-2.36422905, -0.34190802],
       [-2.29938422, -0.59739451],
       [-2.38984217,  0.64683538],
       [-2.07563095,  1.48917752],
       [-2.44402884,  0.0476442 ],
       [-2.23284716,  0.22314807],
       [-2.33464048, -1.11532768],
       [-2.18432817, -0.46901356],
       [-2.1663101 ,  1.04369065],
       [-2.32613087,  0.13307834],
       [-2.2184509 , -0.72867617],
       [-2.6331007 , -0.96150673],
       [-2.1987406 ,  1.86005711],
       [-2.26221453,  2.68628449],
       [-2.2075877 ,  1.48360936],
       [-2.19034951,  0.48883832],
       [-1.898572  ,  1.40501879],
       [-2.34336905,  1.12784938],
       [-1.914323  ,  0.40885571],
       [-2.20701284,  0.92412143],
       [-2.7743447 ,  0.45834367],
       [-1.81866953,  0.08555853],
       [-2.22716331,  0.13725446],
       [-1.95184633, -0.62561859],
       [-2.05115137,  0.24216355],
       [-2.16857717,  0.52714953],
       [-2.13956345,

In [62]:
#вновь разделим выборку на обучающую и тестовую
X_train_PCA, X_test_PCA, y_train_PCA, y_test_PCA = train_test_split(X_reduced, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

In [73]:
#вновь оценим точность алгоритма с тем же количеством соседей, что и до применения PCA
k = 3

y_pred_PCA = knn(X_train_PCA, y_train_PCA, X_test_PCA, k)

print(f'Точность алгоритма после применения PCA при k = {k}: {accuracy(y_pred_PCA, y_test_PCA):.3f}')

Точность алгоритма после применения PCA при k = 3: 0.867


Точность стала ниже..